In [6]:
# CMSC426 Project 3
# GROUP MEMBERS: Andrew Sumner (asumner1), Teimuraz Trapaidze (ttrapaid)
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2
from scipy import ndimage
import math
import random
import sklearn as sk
from sklearn.svm import SVC

In [9]:
folders = ["images/dolphin", "images/airplanes", "images/Leopards"]

def get_imgs_from_folders(folders):
    # Return to this
    # This must be all images, not just dolphin images, and we must track the label
    image_sets = []
    for folder in folders:
        images = []
        i = 0
        for filename in os.listdir(folder):
            i = i + 1
            image = img.imread(os.path.join(folder,filename))
            images.append(image)
            if i >= 65:
                break
        image_sets.append(images)
    return image_sets

all_images = get_imgs_from_folders(folders)

images = all_images[0] # REMOVE THIS LATER!!!!!!!!!!!!!!!!
#print(images)

In [10]:
# supposedly we get descriptors for all images in a category, then 
# stack ? 
desc_all_imgs = []
kp_all_imgs = []

for image in images:
    sift = cv2.SIFT_create()
    kp, descriptors = sift.detectAndCompute(image, None)
    print(descriptors)
    print(descriptors.shape)
    print("----------------------------------")
    desc_all_imgs.append(descriptors)
    kp_all_imgs = kp_all_imgs + kp
    
    break
#     print(descriptors.shape)
#     for pt, desc in zip(kp, descriptors):
#         (x,y) = pt.pt
#         print((x,y), desc)

desc_all_imgs = tuple(desc_all_imgs)
desc_all_imgs = np.vstack(desc_all_imgs)
print(desc_all_imgs)
print()
print(desc_all_imgs.shape)

[[ 68.   3.   5. ...   0.   0. 142.]
 [  1.   4.  24. ...   6.   0.   0.]
 [  0.   0.   0. ...  22.   1.   7.]
 ...
 [ 76.   0.   0. ...  46.   4.   7.]
 [ 29.   0.   0. ...  31.   0.   1.]
 [ 67.   0.   0. ...  25.   9.   5.]]
(331, 128)
----------------------------------
[[ 68.   3.   5. ...   0.   0. 142.]
 [  1.   4.  24. ...   6.   0.   0.]
 [  0.   0.   0. ...  22.   1.   7.]
 ...
 [ 76.   0.   0. ...  46.   4.   7.]
 [ 29.   0.   0. ...  31.   0.   1.]
 [ 67.   0.   0. ...  25.   9.   5.]]

(331, 128)


In [16]:
# this k means clustering method is written for one image
# im not sure how, but were supposed to get it to work with 
# every image? 

# if you look at slide 43 of bovw, it says to set each cluster C_i 
# to be the set of points X that are closer to the center c_i than 
# any other center ... but what is X if were using every feature vector
# from every image? if its one image, X would just be the set of points 
# of each descriptor/keypoint for that particular image, but does it 
# work like that if youre using every image?

# if so, we would just have to pass in keypoints from every image 
# and descriptors from every image instead of just one like i did 
# and this method will probably work

# 428 on piazza seems to be the same question I have, but its unanswered 
# as of writing this

# using slide 43 on bagof visual words lecture

def clustering(keypoints, descriptors, k):
    num_descs = len(keypoints)
    
    centers = []
    center_descs = []
    # initial random k centers
    for i in range(k):
        center = random.randint(0, num_descs-1)
        centers.append((keypoints[center].pt[0], keypoints[center].pt[1]))
        center_descs.append(descriptors[center])
        
    print(len(centers))
    old_centers = [] 
    
    while (old_centers != centers):
        
        clusters = {center:[] for center in centers}
        print(len(clusters.items()))
        for keypoint, descriptor in zip(keypoints, descriptors):
            closest = 99999999999
            for center, center_desc in zip(centers, center_descs):
                dist = np.sum((center_desc-descriptor)**2)
                if dist < closest:
                    closest = dist
                    closest_center = center
            clusters[closest_center].append(np.array([keypoint.pt[0], keypoint.pt[1]]))
        
        old_centers = centers
        centers = []
        
        
        for center, cluster in clusters.items():
            if (len(cluster) > 0):
                cluster = np.vstack(cluster)
                new_center = np.sum(cluster,axis=0)/np.linalg.norm(cluster)
                centers.append((new_center[0], new_center[1]))

            
    return centers

In [17]:
# i havent checked this yet, but if you were to sum the number of key 
# points in each cluster, it should equal the number of descriptors 

# this is for one image

# issue: why is the number of clusters decreasing with each iteration

centers = clustering(kp_all_imgs, desc_all_imgs, 400)

400
392
392
385
378
372
366
360
355
350
345
340
335
330
325
321
317
313
309
305
301
297
293
289
285
281
277
273
269
265
262
259


KeyboardInterrupt: 

In [ ]:
def generate_histogram(descriptors, centers): # descriptors is for a single image
    hist_arr = []
    num_bins = len(centers)
    for row in descriptors:
        min_dist = 9999999999999999999
        min_center_idx = -1
        center_idx = -1
        for center in centers:
            center_idx = center_idx + 1
            dist = np.linalg.norm(row - center)
            if dist < min_dist:
                min_dist = dist
                min_center_idx = center_idx
        hist_arr.append(min_center_idx)
    hist = np.histogram(hist_arr, bins=num_bins)
    plt.hist(hist_arr, bins=num_bins)
    plt.show()
    return hist

In [ ]:
# svm

In [ ]:
# use this later once we have our test and train cleaned up

svclassifier = sk.svm.SVC(kernel='linear')
svclassifier.fit(x_train, y_train)
y_test_pred = svclassifier.predict(x_test)

print(sk.metrics.classification_report(y_test, y_test_pred))